In [1]:
def get_model():
    from tensorflow.keras.models import Sequential
    from keras import initializers
    from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 1), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(320, 320, 1), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='sigmoid',kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer=initializers.Zeros()))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [2]:
from mxnet import np
import os
def fullpath(path, files):
    return  [(lambda x: path + x)(x) for x in files]
val_dir = "../chest_xray/chest_xray/test"
filesPneumo = fullpath(val_dir+'/PNEUMONIA/',os.listdir(os.path.join(val_dir, 'PNEUMONIA')))
del filesPneumo[0]
labelsPneumo = np.zeros(len(filesPneumo))
filesNormal = fullpath(val_dir+'/NORMAL/',os.listdir(os.path.join(val_dir, 'NORMAL')))
del filesNormal[0]
labelsNormal = np.ones(len(filesNormal))
fileData = filesPneumo + filesNormal
fileLabels = np.concatenate((labelsPneumo,labelsNormal),axis=0).astype(np.float32)

In [3]:

model = get_model()


Using TensorFlow backend.


In [4]:
import mxnet as mx
ctx = mx.gpu(0)
from mxnet.gluon import nn, Trainer, loss
from mxnet import optimizer
from mxnet import autograd, np, npx
def convert_keras_to_mxnet(model):
    npx.set_np()
    def layer_conversion(layer):
        t = str(type(layer)).split(".")
        layer_type = t[-1].replace('\'>',"")
        config = layer.get_config()
        if layer_type == "Dense":
            return nn.Dense(units=config['units'], activation=config['activation'], use_bias=config['use_bias'], dtype=np.float32)
        if layer_type == "Conv2D":
            return nn.Conv2D(channels=config['filters'], kernel_size=config['kernel_size'],use_bias=config['use_bias'],activation=config['activation'], weight_initializer=mx.init.Normal(0.01), bias_initializer=mx.init.Zero())
        if layer_type == "MaxPooling2D":
            return nn.MaxPool2D(pool_size=config['pool_size'])
        if layer_type == "BatchNormalization":
            return nn.BatchNorm(momentum=config['momentum'], epsilon=config['epsilon'])
        if layer_type=="Dropout":
            return nn.Dropout(rate=config['rate'])
        if layer_type=="Flatten":
            return nn.Flatten()
        return None
    mxModel = nn.Sequential()
    for layer in model.layers:
        new_layer = layer_conversion(layer)
        mxModel.add(new_layer)
    return mxModel

new_model = convert_keras_to_mxnet(model)



In [5]:
new_model.initialize(force_reinit=True)

In [6]:
from mxnet import image
def preprocess(img,label):
    img = image.imresize(img,320,320)
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :].astype(np.float32)
    return img,label
ds2 =  mx.gluon.data.vision.datasets.ImageFolderDataset("../chest_xray/chest_xray/train",transform=preprocess)




/home/andrea/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Ignoring ../chest_xray/chest_xray/train/.DS_Store, which is not a directory.
  
/usr/local/lib/python3.6/dist-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../chest_xray/chest_xray/train/NORMAL/.DS_Store of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))
/usr/local/lib/python3.6/dist-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../chest_xray/chest_xray/train/PNEUMONIA/.DS_Store of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))


In [7]:
def get_batches(dataset, batch_size, labels):
    num_steps = len(dataset)//batch_size
    steps = range(num_steps)
    data_batches = []
    label_batches = []
    for step in steps:
        batch_idx = range(batch_size) if len(dataset.items[step*batch_size:])>batch_size else range(len(dataset.items[step*batch_size:]))
        for idx in batch_idx:
            if idx == 0:
                img_arr = dataset[step*batch_size+idx][0]

            else:
                img_arr = np.concatenate((img_arr,dataset[step*batch_size+idx][0]),axis=0)
        label_batches.append(labels[step*batch_size:step*batch_size+len(batch_idx)])
        data_batches.append(img_arr)
    return  data_batches, label_batches

In [8]:
data_batches, label_batches = get_batches(ds2, 8, fileLabels)

In [24]:
y_true = np.array([0.,0.,1.,1.]).reshape(-1,1)

In [ ]:
params = new_model[-1].collect_params()

In [ ]:
dense_W = params.get("weight").data()
dense_B = params.get("bias").data()

In [ ]:
dense_W.attach_grad()
dense_B.attach_grad()


In [ ]:
dense_W

In [27]:
trainer=mx.gluon.Trainer(new_model.collect_params(),'adam', {'learning_rate': 0.001})

In [25]:
images = ds2[0][0]
images = np.concatenate((images,ds2[1][0], ds2[-1][0], ds2[-2][0]), axis=0)

In [46]:
with autograd.record():
    y_ =  new_model.forward(images)
    lossValue = BCE(y_, y_true)
y_[y_<0.5]=0
y_[y_>=0.5]=1
acc = (sum(y_==y_true)/len(y_true))*100
print( "acc: " + str(acc) + "; Loss: " + str(lossValue))

acc: [100.]; Loss: [0.69314724 0.69314724 0.31326175 0.31326175]


In [ ]:
def train(dataset_batches, labels, epochs, learning_rate):
    trainer = mx.gluon.Trainer(new_model.collect_params(),'adam', {'learning_rate': learning_rate})
    BCE = loss.SigmoidBinaryCrossEntropyLoss()
    epochs = range(epochs)
    for epoch in epochs:
        for data_batch, labels_batch in zip(dataset_batches, labels):
            y_true = labels_batch.reshape(-1,1)
            with autograd.record():
                y_ =  new_model.forward(data_batch)
                lossValue = BCE(y_, y_true)
            lossValue.backward()
            trainer.step(data_batch.shape[0])
        y_[y_<0.5]=0
        y_[y_>=0.5]=1
        acc = (sum(y_==y_true)/len(y_true))*100
        print( "Epoch: "+ str(epoch+1) +"; "+ "acc: " + str(acc) + "; Loss: " + str(lossValue))

In [ ]:
train(data_batches,label_batches, 10, 0.001)

In [15]:
bl = label_batches[-1]

In [16]:
bl2 = bl.reshape(-1,1)

In [24]:
del data_batches, label_batches